<a href="https://colab.research.google.com/github/shobhits07/docs/blob/main/Landsat_9_All_Bands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install geemap

In [2]:
import geemap
import ee

In [ ]:
# Authenticate and initialize
ee.Authenticate()
ee.Initialize()

In [ ]:
# Display the map
Map=geemap.Map()
Map

In [ ]:
# Define the area of interest
roi_rect = ee.FeatureCollection('projects/ee-shobhit007/assets/MP_Districts').geometry()

# Load the Landsat 9 Image Collection for Tier 1 Level 2 data
dataset = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterDate('2022-07-01', '2022-12-01') \
    .filterBounds(roi_rect)  # Filter by Area of Interest

# Applies scaling factors to the image bands
def applyScaleFactors(image):
    # Scale and offset for optical bands
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

    # Scale and offset for thermal bands
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Add scaled bands to the image
    return image.addBands(opticalBands, None, True) \
                .addBands(thermalBands, None, True)

# Apply the scaling factors to the entire dataset
dataset = dataset.map(applyScaleFactors)

# Clip the dataset by the AOI
clippedDataset = dataset.map(lambda image: image.clip(roi_rect))

# Reduce the clipped dataset to a single image (e.g., median)
compositeImage = clippedDataset.median()

# Visualization parameters for the True Color image
vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # Bands for RGB-Natural color visualization
    'min': 0.0,  # Minimum pixel value for visualization
    'max': 0.3,  # Maximum pixel value for visualization
}

# Center the map on the area of interest
Map.centerObject(roi_rect)

# Add the AOI boundary to the map
Map.addLayer(roi_rect, {'color': 'red'}, 'AOI Boundary')

# Add the clipped dataset to the map with the specified visualization parameters
Map.addLayer(compositeImage, vis_params, 'STD False Color (543)')

Map


In [44]:
# Export the map to Google Drive
geemap.ee_export_image_to_drive(
    image=compositeImage,
    description='Landsat_9_export_to_drive',
    folder='Landsat_9_exports',
    region=roi_rect,
    scale=300
)